## **Installing Dependencies**

In [ ]:
!pip install datasets bitsandbytes torch transformers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 81.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `thesis` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `thesis`


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import warnings
warnings.filterwarnings("ignore")

## **Importing Libraries**

In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import numpy as np
import torch
import ast
import random
from collections import Counter
import re
import pandas as pd

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    precision_score,
    recall_score,
    classification_report
)

## **Load the Data**

In [ ]:
# Considers only the domain related to science and world knowledge

science = load_dataset('hkust-nlp/felm', 'science', trust_remote_code=True)
wk = load_dataset('hkust-nlp/felm', 'wk', trust_remote_code=True)

## **Data Preparation**

In [ ]:
# Concatenate the data from the domains

hallu_felm = concatenate_datasets([science['test'], wk['test']])

In [ ]:
# Label each instances: if all the segmented responses are true, then the full passage is true, on contrary if at least one segmented response is false,
# then the entire passage is false

def compute_new_column(example):
    return {"hallucination": False if False in example["labels"] else True}

hallu_felm = hallu_felm.map(compute_new_column)

Map:   0%|          | 0/309 [00:00<?, ? examples/s]

In [ ]:
# Convert to Pandas for better handling

hallu_felm = hallu_felm.to_pandas()

In [ ]:
# Remove instances without the response

hallu_felm = hallu_felm[~hallu_felm['response'].isna()].reset_index()

In [ ]:
# Function that filters the questions based on if the prompt ends with a question mark

def is_question(prompt):
    if prompt.strip().endswith("?"):
        return True
    else:
        return False


hallu_felm['is_question'] = hallu_felm['prompt'].apply(is_question)

In [ ]:
# Filter the questions

hallu_felm = hallu_felm[hallu_felm['is_question'] == True].reset_index(drop=True)

In [ ]:
# Choose 50 random examples from the True class and 50 random examples from the False class, in order to balance the dataset

random.seed(777)

num_classes = hallu_felm["hallucination"].nunique()
samples_per_class = 100 // num_classes

hallu_felm_sample = hallu_felm.groupby("hallucination").sample(n=samples_per_class, random_state=42)

# Set the dataset for the examples to be shown to the LLM as the ones that were not chosen previously
hallu_felm_examples = hallu_felm.drop(hallu_felm_sample.index).reset_index(drop=True)

In [ ]:
# Consider only the prompt, the response and the factuality label

hallu_felm_sample = hallu_felm_sample[['prompt', 'response', 'hallucination']]

In [ ]:
# Check for the correctness of the rebalance

Counter(hallu_felm_sample['hallucination'])

Counter({False: 50, True: 50})

In [ ]:
random.seed(777)

hallu_felm_sample = Dataset.from_pandas(hallu_felm_sample).shuffle(42) # Convert to Dataset format
hallu_felm_sample[0]

{'prompt': 'This Jewish American studied at NYU, was born and lived in New York City, and became famous in the mid 1980s for a controversial act. His name is "Bernie" what?',
 'response': 'The person you are referring to is Bernie Goetz, also known as the "Subway Vigilante".',
 'hallucination': True,
 '__index_level_0__': 233}

In [ ]:
hallu_felm_examples = hallu_felm_examples[['prompt', 'response', 'hallucination']] # Consider only the prompt, the response and the factuality label
hallu_felm_examples = Dataset.from_pandas(hallu_felm_examples) # Convert to Dataset format

## **Load the model**

In [ ]:
# Configures bitsandbytes for a 4-bit quantization of the LLM with double quantization, for the efficiency
# load_in_4bit: Enables the 4-bit quantization of the model
# bnb_4bit_use_double_quant: Enables double quantization that further decreases the computational efforts
# bnb_4bit_quant_type: Type of quantization used
# bnb_4bit_compute_dtype: Define the type of data used during computation in the inference phase

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [ ]:
# Retrieve the model and the corresponding tokenizer from Hugging Face using the method AutoModelForCausalLM to use the generative part of the LLM

model_id = 'meta-llama/Llama-3.2-3B-Instruct'

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config=bnb_config,
                                             device_map='auto')

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/878 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
# Text generation pipeline

# max_new_tokens: Maximum number of new tokens generated
# temperature: Controls the randomness of the new tokens
# truncation: if the input is too long, it gets truncated to respect the maximum limit of tokens supported by the LLM

text_generator = pipeline(task="text-generation",
                          model=model,
                          tokenizer=tokenizer,
                          do_sample = False,
                          max_new_tokens=128,
                          temperature = 1e-3,
                          truncation = True)

Device set to use cuda:0


In [ ]:
# Function to get the text generated by the LLM

def get_response(prompt):
  return text_generator(prompt,
                        pad_token_id=text_generator.tokenizer.eos_token_id, return_full_text = False)[0]['generated_text']

## **Chat Building**



In [ ]:
def build_chat(k = 1):

  # Prompt to instruct the LLM for the task

  messages = [{
      'role': 'system',
      'content': f"""I want you to act as a response judge. Given a user query and a response by an LLM, your objective is to determine if the response is an hallucination or not.
    In the context of NLP, an "hallucination" refers to a phenomenon where the LLM generates text that is incorrect, nonsensical, or not real.
    Based on your knowledge and on the definition of hallucination provided, analyze the user query and the response of TLLM, and answer the following question: is the response factual or not?
    Answer True if you consider the response factual, False otherwise. You don't have to provide any explanation."""}]

  data = hallu_felm_examples

  # Add the examples to the list of messages

  for i in range(k):
    demo = {
        'role':'user',
        'content': f"""User query: {data['prompt'][i]}
LLM response: {data['response'][i]}"""
    }
    result = {
        'role':'assistant',
        'content': f"""Answer: {data['hallucination'][i]}"""
    }
    messages.append(demo)
    messages.append(result)

  return messages

## **One-Shot Classification**



In [ ]:
def one_shot_classification(example):

# Builds the prompt with one example

  system_message = build_chat()

# Add the response to be classified to the list of messages, along with the corresponding prompt

  question = {
      'role':'user',
      'content': f"""User query: {example['prompt']}
                     LLM response: {example['response']}"""
  }
  system_message.append(question)
  prompt = tokenizer.apply_chat_template(system_message, tokenize = False, add_generation_prompt = True) # Transform the list of messages in a chat template readable by the LLM
  result = get_response(prompt) # Extract the response from the LLM
  result = result.split('Answer:')[-1].strip() # Post processing of the output to extract the response that we are interested in (True or False)
  return {'prediction': result} # Save the prediction in a new column of the dataset

In [ ]:
print(tokenizer.apply_chat_template(build_chat(1), tokenize = False, add_generation_prompt = True)) # Example of chat template

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 12 Feb 2025

I want you to act as a response judge. Given a user query and a response by an LLM, your objective is to determine if the response is an hallucination or not.
    In the context of NLP, an "hallucination" refers to a phenomenon where the LLM generates text that is incorrect, nonsensical, or not real.
    Based on your knowledge and on the definition of hallucination provided, analyze the user query and the response of TLLM, and answer the following question: is the response factual or not?
    Answer True if you consider the response factual, False otherwise. You don't have to provide any explanation.<|eot_id|><|start_header_id|>user<|end_header_id|>

User query: Have humans flown close to the sun?
LLM response: No, humans have never flown close to the sun. The sun's surface temperature is about 9,900 degrees Fahrenheit (5,500 degrees Celsius), which is hot enough

In [ ]:
# Start the one shot prompting

hallu_felm_sample = hallu_felm_sample.map(lambda x: one_shot_classification(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
def compute_metrics(y_true, y_pred):

# Function that computes the metrics with the predictions and the true labels

  metrics = {
      "accuracy": accuracy_score(y_true, y_pred),
      "precision": precision_score(y_true, y_pred, average="binary"),
      "recall": recall_score(y_true, y_pred, average="binary"),
      "macro_f1": f1_score(y_true, y_pred, average="binary"),
      "micro_f1": f1_score(y_true, y_pred, average="binary")
  }
  return metrics

In [ ]:
# Function that extracts True of False from the response using a regex

def response(example):
  pattern = r'.*(True|False)'
  match = re.search(pattern, example['prediction'])
  if match:
    match_found = match.group(1)
    example['prediction'] = ast.literal_eval(match_found)

  return example

In [ ]:
# Extracts True or False

hallu_felm_sample = hallu_felm_sample.map(lambda x: response(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Compute the metrics

one_shot_metrics = compute_metrics(hallu_felm_sample['hallucination'], hallu_felm_sample['prediction'])

In [ ]:
one_shot_metrics

{'accuracy': 0.62,
 'precision': 0.6071428571428571,
 'recall': 0.68,
 'macro_f1': 0.6415094339622641,
 'micro_f1': 0.6415094339622641}

## **5-Shot Classification**

In [ ]:
def five_shot_classification(example):

# Builds the prompt with 5 examples

  system_message = build_chat(5)

# Add the response to be classified to the list of messages, along with the corresponding prompt

  question = {
      'role':'user',
      'content': f"""User query: {example['prompt']}
                     LLM response: {example['response']}"""
  }
  system_message.append(question)
  prompt = tokenizer.apply_chat_template(system_message, tokenize = False, add_generation_prompt = True) # Transform the list of messages in a chat template readable by the LLM
  result = get_response(prompt) # Extract the response from the LLM
  result = result.split('Answer:')[-1].strip() # Post processing of the output to extract the response that we are interested in (True or False)
  return {'five_shot_prediction': result} # Save the prediction in a new column of the dataset

In [ ]:
# Start the 5 shot prompting

hallu_felm_sample = hallu_felm_sample.map(lambda x: five_shot_classification(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Function that extracts True of False from the response using a regex

def response_5shot(example):
  pattern = r'.*(True|False)'
  match = re.search(pattern, example['five_shot_prediction'])
  if match:
    match_found = match.group(1)
    example['five_shot_prediction'] = ast.literal_eval(match_found)

  return example

In [ ]:
# Extracts True or False from the response

hallu_felm_sample = hallu_felm_sample.map(lambda x: response_5shot(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Compute the metrics

five_shot_metrics = compute_metrics(hallu_felm_sample['hallucination'], hallu_felm_sample['five_shot_prediction'])

In [ ]:
five_shot_metrics

{'accuracy': 0.56,
 'precision': 0.625,
 'recall': 0.3,
 'macro_f1': 0.40540540540540543,
 'micro_f1': 0.40540540540540543}

In [ ]:
def ten_shot_classification(example):

# Builds the prompt with 10 examples

  system_message = build_chat(10)

# Add the response to be classified to the list of messages, along with the corresponding prompt

  question = {
      'role':'user',
      'content': f"""User query: {example['prompt']}
                     LLM response: {example['response']}"""
  }
  system_message.append(question)
  prompt = tokenizer.apply_chat_template(system_message, tokenize = False, add_generation_prompt = True) # Transform the list of messages in a chat template readable by the LLM
  result = get_response(prompt) # Extract the response from the LLM
  result = result.split('Answer:')[-1].strip() # Post processing of the output to extract the response that we are interested in (True or False)
  return {'ten_shot_prediction': result} # Save the prediction in a new column of the dataset

In [ ]:
# Starts the ten shot classification

hallu_felm_sample = hallu_felm_sample.map(lambda x: ten_shot_classification(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Function that extracts True of False from the response using a regex

def response_10shot(example):
  pattern = r'.*(True|False)'
  match = re.search(pattern, example['ten_shot_prediction'])
  if match:
    match_found = match.group(1)
    example['ten_shot_prediction'] = ast.literal_eval(match_found)

  return example

In [ ]:
# Extracts True or False from the response

hallu_felm_sample = hallu_felm_sample.map(lambda x: response_10shot(x))

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
# Computes the metrics

ten_shot_metrics = compute_metrics(hallu_felm_sample['hallucination'], hallu_felm_sample['ten_shot_prediction'])

In [ ]:
ten_shot_metrics

{'accuracy': 0.59,
 'precision': 0.5849056603773585,
 'recall': 0.62,
 'macro_f1': 0.6019417475728155,
 'micro_f1': 0.6019417475728155}

## **Saving dataframes with metrics**

Convert the results in a dataframe and save it on Google Drive.

In [ ]:
path = '/content/drive/MyDrive/Ceccarelli_MasterThesis&Internship/Master Thesis/Results'

In [ ]:
metrics = {
    'classification type' : ['one shot', 'five shot', 'ten shot'],
    'accuracy' : [one_shot_metrics['accuracy'], five_shot_metrics['accuracy'], ten_shot_metrics['accuracy']]
}

metrics_df = pd.DataFrame(metrics)
metrics_df

,classification type,accuracy
0,one shot,0.62
1,five shot,0.56
2,ten shot,0.59


In [ ]:
with open(path + "/metrics_shot_classification_felm.csv", "w") as f:
    metrics_df.to_csv(f, index=False)